# Libraries and Dependencies

In [45]:
# uncomment for the first run!

# !pip install spacy
# !pip install spacy-entity-linker
# !python -m spacy download en_core_web_sm
# !python -m spacy download en_core_web_lg
# !python -m spacyEntityLinker "download_knowledge_base"

# NOTE: Restart the runtime after running this cell.

# Imports

In [49]:
import spacy
import pandas as pd
import json
import numpy as np
from spacy_entity_linker import EntityLinker
import os


# Data Loading

In [ ]:
# Load the data
trainData = pd.read_json("./train.jsonl", lines=True, orient="records")
valData = pd.read_json("./val.jsonl", lines=True, orient="records")
testData = pd.read_json("./test.jsonl", lines=True, orient="records")

# Train 
trainPassages = trainData.passage.values
trainQuestions = trainData.question.values
trainAnswers = trainData.label.values.astype(int)

# Validation
valPassages = valData.passage.values
valQuestions = valData.question.values
valAnswers = valData.label.values.astype(int)

# Test
testPassages = testData.passage.values
testQuestions = testData.question.values


# FinalSet
finalPassages = np.concatenate((trainPassages, valPassages))
finalPassages = np.concatenate((finalPassages, testPassages))

finalQuestions = np.concatenate((trainQuestions, valQuestions))
finalQuestions = np.concatenate((finalQuestions, testQuestions))


2021-05-09 19:57:28,452 - numexpr.utils - INFO - NumExpr defaulting to 2 threads.


In [ ]:
#Initialize Entity Linker
entityLinker = EntityLinker()

#initialize language model
nlp = spacy.load('en_core_web_sm')

#add pipeline
nlp.add_pipe(entityLinker, last=True, name="entityLinker")


# Entity Linking & Relation Extraction

In [ ]:
model = on.get_model('wiki80_bert_softmax')


2021-05-09 19:58:49,051 - root - INFO - Loading BERT pre-trained checkpoint.


In [ ]:
model1 = on.get_model('wiki80_bertentity_softmax')

2021-05-09 20:02:09,686 - root - INFO - Loading BERT pre-trained checkpoint.


In [ ]:
def augmentData(questions, passages, labels, file=''):

  augData = open("Augmented_" + file + ".jsonl", 'w')

  for question, passage, label in zip(questions, passages, labels):
    
    qpDict = {}

    qpDict['question'] = question

    # This gives all the entities present in the question and passage
    q = nlp(question)
    p = nlp(passage)

    #returns all linked entities in the question and passage
    qEnts = q._.linkedEntities
    pEnts = p._.linkedEntities

    if len(qEnts):

      for qe in qEnts:
        
        if qe.get_label() != None:
          if qe.get_label() in passage:
            qeDes = qe.get_description()
          
            if qeDes not in passage:
              passage = passage + qeDes

      qpDict['passage'] = passage
      qpDict['label'] = int(label)

      augData.write(json.dumps(qpDict) + '\n')

In [ ]:
augmentData(trainQuestions, trainPassages, trainAnswers, 'train')
augmentData(valQuestions, valPassages, valAnswers, 'val')

## Nothing helpful from below

BERTSIDE didn't work at all

OpenNRE didnt give out any good relations on our dataset.
Most of the predicted relations were E1 "part of" E2.

The third relation extractor was an implementation of matching the blanks paper fine-tuned on SemEval. Didn't give out any good relations on our data.

## OpenNRE

In [ ]:
# !git clone https://github.com/thunlp/OpenNRE.git

# %cd OpenNRE/
# !pip install -r requirements.txt
# !python setup.py install 

# import opennre as on

In [ ]:
def augmentDataRE():
   # print(question)
    # print(passage)

    # sents = passage.split('.')
    # # print(sents)

    # # check whether the question has any entities
    # if len(qEnts) > 0:

    #   # for every entity in question
    #   for qe in qEnts:
      
    #     qe = qe.get_label()
    #     qeLen = len(qe)

    #     # perform relation extraction of entities present in both question and passage
    #     # A entity which is present in both question and passage is used to find relations with all other entities of the passage
    #     if qe in passage:

    #       # perform sentential relation extraction using every sentence of the passage
    #       # for sent in sents:


    #       for pe in pEnts:
    #         pe = pe.get_label()
    #         # if pe in sent and qe in sent and pe != qe:
              
    #         qeStart = passage.index(qe)
    #         qeEnd = qeStart + qeLen - 1

    #         peLen = len(pe)
    #         if pe in passage and pe != qe:
    #           peStart = passage.index(pe)
    #           peEnd = peStart + peLen - 1

    #           hPos = {'pos': (qeStart, qeEnd)}
    #           tPos = {'pos': (peStart, peEnd)}

              
    #           pred, conf = (model1.infer({'text': passage, 'h': hPos, 't': tPos}))

    #           if conf > 0.85:
    #             print("Qe: %s, Pe: %s" % (qe,pe))
    #             print(pred,conf)
    #   break


## BERT Relation Extraction using Matching the blanks paper

In [ ]:
# !git clone https://github.com/plkmo/BERT-Relation-Extraction.git

In [ ]:
# !wget https://github.com/sahitya0000/Relation-Classification/raw/master/corpus/SemEval2010_task8_all_data.zip


In [11]:
# !unzip SemEval2010_task8_all_data.zip
# %rm -rf sample_data/

In [ ]:
# !pip install seqeval
# !pip install boto3
# !pip install filelock
# !pip install sentencepiece
# %cd BERT-Relation-Extraction/
!python main_task.py \
# --train 0 \
--infer 1 \
--num_epochs 3 \
--model_no 0 \
--model_size bert-base-uncased \


In [ ]:
import pickle as pk
import pandas as pd
# import json
%cd BERT-Relation-Extraction/
argsDict = {}

argsDict = {'model_no': 0, 'num_classes': 19, 'model_size': 'bert-base-uncased'}

filename = "./data/args.jsonl"

outfile = open(filename, 'w')
outfile.write(json.dumps(argsDict) + '\n')
outfile.close()

p = pd.read_json("./data/args.jsonl", lines=True, orient="records")

from src.tasks.infer import infer_from_trained
inferer = infer_from_trained(args=None, detect_entities=True)

In [ ]:
test2 = "[E1]University of New Hampshire[/E1] is in [E2]USA[/E2]"
inferer.infer_sentence(test2, detect_entities=False)